In [5]:
import json

In [144]:
original = json.load(open("data/verses_clauses_dict.json", "r", encoding="utf-8"))
results = [json.loads(line) for line in open("data/fine_tuning_datasets/basic_models_outputs/output_validation_small_gpt4o.jsonl", encoding="utf-8").readlines()]

In [145]:
def extract_result(line):
    return line["verse"].split(":")[1].strip(), line["parsed_clauses"].replace("```", "").replace("json", "").replace("\n", "")

In [146]:
len(results)

200

In [147]:
#extract_result(a)

In [148]:
def extract_results(results):
    verse_clauses = {}
    for result in results:
        verse, clauses = extract_result(result)
        verse_clauses[verse] = clauses
    return verse_clauses

In [149]:
verse_clauses_results = extract_results(results)

In [150]:
#verse_clauses_results

In [151]:
len(verse_clauses_results)

200

In [152]:
nb_total = 0
nb_correct = 0

nb_simple_total = 0
nb_simple_correct = 0

nb_complex_total = 0
nb_complex_correct = 0

nb_clauses_total = 0
nb_clauses_correct = 0

nb_complex_clauses_total = 0
nb_complex_clauses_correct = 0

nb_simple_clauses_total = 0
nb_simple_clauses_correct = 0

for verse, clauses in verse_clauses_results.items():
    nb_total += 1
    true_clauses = original[verse]["clauses"]
    nb_clauses_total += len(true_clauses)
    complexity = original[verse]["complexity"]
    if complexity == "simple":
        nb_simple_clauses_total += len(true_clauses)
        nb_simple_total += 1
    else:
        nb_complex_clauses_total += len(true_clauses)
        nb_complex_total += 1
        
    try:
        clauses = json.loads(clauses)
    except json.JSONDecodeError:
        print("wrong json", clauses)
        continue
    complexity = original[verse]["complexity"]
    print("res", clauses)
    print("cor", true_clauses, complexity)
    print(f" Model result is {clauses == true_clauses}")
    print("")

    for clause in clauses:
        if clause in true_clauses:
            nb_clauses_correct += 1
            
            if complexity == "simple":
                nb_simple_clauses_correct += 1
            else:
                nb_complex_clauses_correct += 1
    
    if clauses == true_clauses:
        nb_correct += 1
        if complexity == "simple":
            nb_simple_correct += 1
        else:
            nb_complex_correct += 1

res ['והיהודים אשרבשושן נקהלו בשלשה עשר בו', 'ובארבעה עשר בו', 'ונוח בחמשה עשר בו', 'ועשה אתו יום משתה ושמחה']
cor ['והיהודים נקהלו בשלשה עשר בו ובארבעה עשר בו', 'אשרבשושן', 'ונוח בחמשה עשר בו', 'ועשה אתו יום משתה ושמחה'] complex
 Model result is False

res ['ויצא משה לקראת חתנו', 'וישתחו', 'וישקלו', 'וישאלו אישלרעהו לשלום', 'ויבאו האהלה']
cor ['ויצא משה', 'לקראת חתנו', 'וישתחו', 'וישקלו', 'וישאלו לשלום', 'אישלרעהו', 'ויבאו האהלה'] complex
 Model result is False

res ['ויצג אתהמקלות אשר פצל ברהטים בשקתות המים', 'אשר תבאן הצאן לשתות לנכח הצאן', 'ויחמנה בבאן לשתות']
cor ['ויצג אתהמקלות ברהטים בשקתות המים לנכח הצאן', 'אשר פצל', 'אשר תבאן הצאן', 'לשתות', 'ויחמנה', 'בבאן', 'לשתות'] complex
 Model result is False

res ['ויגד משה לאהרן את כלדברי יהוה אשר שלחו', 'ואת כלהאתת אשר צוהו']
cor ['ויגד משה לאהרן את כלדברי יהוה ואת כלהאתת', 'אשר שלחו', 'אשר צוהו'] complex
 Model result is False

res ['ומכה נפשבהמה ישלמנה', 'נפש תחת נפש']
cor ['וישלמנה נפש תחת נפש', 'מכה נפשבהמה'] complex
 Model result

In [153]:
print(f"Correct parsing: {nb_correct} over {nb_total}.")
print(f"Percentage of correct parsing: {nb_correct / nb_total * 100} %")

Correct parsing: 51 over 200.
Percentage of correct parsing: 25.5 %


In [154]:
print(f"Correct parsing of simple verses: {nb_simple_correct} over {nb_simple_total}.")
print(f"Percentage of correct parsing for simple verses: {nb_simple_correct / nb_simple_total * 100} %")

Correct parsing of simple verses: 51 over 192.
Percentage of correct parsing for simple verses: 26.5625 %


In [155]:
print(f"Correct parsing of complex verses: {nb_complex_correct} over {nb_complex_total}.")
print(f"Percentage of correct parsing for complex verses: {nb_complex_correct / nb_complex_total * 100} %")

Correct parsing of complex verses: 0 over 8.
Percentage of correct parsing for complex verses: 0.0 %


In [156]:
print(f"Correct parsing of individual clauses: {nb_clauses_correct} over {nb_clauses_total}.")
print(f"Percentage of correct parsing: {nb_clauses_correct / nb_clauses_total * 100} %")

Correct parsing of individual clauses: 313 over 747.
Percentage of correct parsing: 41.900937081659976 %


In [157]:
print(f"Correct parsing of individual clauses in simple verses: {nb_simple_clauses_correct} over {nb_simple_clauses_total}.")
print(f"Percentage of correct parsing in simple verses: {nb_simple_clauses_correct / nb_simple_clauses_total * 100} %")

Correct parsing of individual clauses in simple verses: 305 over 707.
Percentage of correct parsing in simple verses: 43.14002828854314 %


In [158]:
print(f"Correct parsing of individual clauses in complex verses: {nb_complex_clauses_correct} over {nb_complex_clauses_total}.")
print(f"Percentage of correct parsing in complex verses: {nb_complex_clauses_correct / nb_complex_clauses_total * 100} %")

Correct parsing of individual clauses in complex verses: 8 over 40.
Percentage of correct parsing in complex verses: 20.0 %
